In [144]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from collections import defaultdict
import re
import numpy as np
import pandas as pd
import string
from datetime import datetime

chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [18]:
url = 'http://www.marketwatch.com'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [19]:
#Search 'yellen'
path = '//a[@class="btn btn--outline btn--search"]'
driver.find_element_by_xpath(path).click()
driver.find_element_by_xpath('//a[text()="Advanced Search"]').send_keys(Keys.RETURN)

#Advanced Searcb
driver.find_element_by_id('refinesearchtoggle').click()
driver.find_element_by_xpath('//input[@type="checkbox"]').click()

#Set Subject
search_mode = '//select[@id="mp"]/option[@value="806"]'
driver.find_element_by_xpath(search_mode).click()

#Results Per Page
results = '//select[@id="rpp"]/option[@value="100"]'
driver.find_element_by_xpath(results).click()

#Set date
date = driver.find_element_by_id('bdv')
date.click()
date.send_keys('09/30/2017')

#Search
driver.find_element_by_xpath('//input[@value="Search"]').click()
current_url = driver.current_url

In [379]:
current_url = 'http://www.marketwatch.com/search?q=&m=Keyword&rpp=100&mp=806&bd=true&bd=false&bdv=09%2F30%2F2017&rs=true'
soup = BeautifulSoup(requests.get(current_url).text, 'html.parser')
soup.find(class_='searchresult').a['href']


'/story/the-worlds-cryptocurrencies-are-bigger-than-paypal-expedia-zillow-and-twitter-combined-2017-09-29'

In [89]:
links = dict()
month = 'Sept'
while 'Sept' in month:
    soup = BeautifulSoup(requests.get(current_url).text, 'html.parser')
    pages = soup.find_all(class_='searchresult')
    for div in pages:
        attrs = dict()
        if div.find('a', href=True):
            if div.a.parent.name == 'div':
                attrs['url'] = div.a['href']
                attrs['date'] = div.next_sibling.span.text
                links[div.a.text] = attrs
        if div == pages[-1]:
            if 'Sept' in pages[0].next_sibling.span.text:
                driver = webdriver.Chrome(chromedriver)
                driver.get(current_url)
                driver.find_element_by_xpath('//a[text()="Next"]').click()
                current_url = driver.current_url
            else:
                month = ''

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[text()="Next"]"}
  (Session info: chrome=61.0.3163.100)
  (Driver info: chromedriver=2.32.498537 (cb2f855cbc7b82e20387eaf9a43f6b99b6105061),platform=Mac OS X 10.12.6 x86_64)


In [91]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
regex.sub('', k)

The world’s cryptocurrencies are bigger than PayPal Expedia Zillow and Twitter combined
SEC charges two initial coin offerings with fraud
Activist Nelson Peltz wins more backing in battle against Procter  Gamble
AIG no longer a threat to US financial stability FSOC says
The Americans most likely to fear financial crisis and how this woman paid off 20000 in debt
Trump Today President accepts Price’s resignation after privatejet outrage
How a failed Alzheimer’s drug stung some large hedge funds
What I’ve learned over 14 years of covering the ‘depressing’ — but crucial — topic of retirement
What to do if you’re suddenly out of a job like Tom Price Steve Bannon James Comey or Anthony Scaramucci
The Kurds meet all the criteria to have their own country
More evidence coffee drinkers live longer — even those ordering decaf
Tom Price out as HHS secretary after privatejet scandal
Trump says he will make a decision on top Fed job in 2 to 3 weeks
Treasury yields see big September rise on hawkish 

In [96]:
links

{'The world’s cryptocurrencies are bigger than PayPal, Expedia, Zillow and Twitter combined': {'date': '6:40 p.m. Sept. 29, 2017',
  'url': '/story/the-worlds-cryptocurrencies-are-bigger-than-paypal-expedia-zillow-and-twitter-combined-2017-09-29'},
 'SEC charges two initial coin offerings with fraud': {'date': '6:33 p.m. Sept. 29, 2017',
  'url': '/story/sec-charges-two-initial-coin-offerings-with-fraud-2017-09-29'},
 'Activist Nelson Peltz wins more backing in battle against Procter & Gamble': {'date': '6:24 p.m. Sept. 29, 2017',
  'url': '/story/activist-nelson-peltz-wins-more-backing-in-battle-against-procter-gamble-2017-09-29'},
 'AIG no longer a threat to U.S. financial stability, FSOC says': {'date': '6:22 p.m. Sept. 29, 2017',
  'url': '/story/aig-no-longer-a-threat-to-us-financial-stability-fsoc-says-2017-09-29'},
 'The Americans most likely to fear financial crisis and how this woman paid off $20,000 in debt': {'date': '5:36 p.m. Sept. 29, 2017',
  'url': '/story/the-americans

In [434]:
df = pd.DataFrame()

In [435]:
dictlist = [x for x in links.items()]

In [436]:
df['article'] = [x[0].lower() for x in dictlist]

In [437]:
df['datetime'] = [x[1]['date'] for x in dictlist]
df['url'] = [x[1]['url'] for x in dictlist]

In [438]:
regex = re.compile('[%s]' % re.escape(string.punctuation.replace(':', '')))
df['datetime'] = df['datetime'].apply(lambda x: regex.sub('', x))

In [439]:
df['time'] = df['datetime'].apply(lambda x: re.search(r'^(.+?\s.+?)\s', x)[1])

In [440]:
df['date'] = df['datetime'].apply(lambda x: re.search(r'([A-Z].*)', x)[1])
df['date'] = df['date'].apply(lambda x: re.sub(r'([A-Za-z]{3})[a-z]*', r'\1', x))

In [441]:
df['datetime'] = df['date'] + ' ' + df['time']

In [442]:
df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, '%b %d %Y %I:%M %p'))

In [443]:
df['url'] = 'http://marketwatch.com' + df['url']

In [444]:
df = df.sort_values(['date', 'datetime']).set_index('date')

In [445]:
df

,article,datetime,url,time
date,,,,
Aug 8 2017,most of what you think about inequality is wrong,2017-08-08 10:44:00,http://marketwatch.com/story/why-the-story-of-...,10:44 am
Mar 27 2017,think you would win an argument with a robot? ...,2017-03-27 10:30:00,http://marketwatch.comhttp://www.marketwatch.c...,10:30 am
May 25 2017,these startups will help you make a down payme...,2017-05-25 14:05:00,http://marketwatch.com/story/as-down-payment-n...,2:05 pm
Sep 1 2017,why aren’t there enough houses to buy?,2017-09-01 08:47:00,http://marketwatch.com/story/why-arent-there-e...,8:47 am
Sep 14 2017,here’s who to start and sit in week 2 of fanta...,2017-09-14 12:19:00,http://marketwatch.com/story/heres-who-to-star...,12:19 pm
Sep 14 2017,chloe the mini frenchie's owner on how to make...,2017-09-14 13:57:00,http://marketwatch.comhttp://www.marketwatch.c...,1:57 pm
Sep 15 2017,orrin hatch says corporate-tax rate could go a...,2017-09-15 05:17:00,http://marketwatch.com/story/orrin-hatch-says-...,5:17 am
Sep 15 2017,assailant arrested after knife attack on soldi...,2017-09-15 06:05:00,http://marketwatch.com/story/assailant-arreste...,6:05 am
Sep 15 2017,this precursor to gin will have you wanting to...,2017-09-15 06:14:00,http://marketwatch.com/story/this-precursor-to...,6:14 am
